# Aula 05 - Avaliação de Sistemas de Recomendação - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar.gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [3]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [4]:
movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))
movies_genres.head()

,movieId,title,genre
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary
2,37,Across the Sea of Time (1995),IMAX


In [5]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [6]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}

df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
movies_tags.dropna(inplace=True)
movies_tags['movieId'] = movies_tags.movieId.astype(int)
movies_tags['userId'] = movies_tags.userId.astype(int)

movies_genres['movieId'] = movies_genres['movieId'].map(map_items)
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres.movieId.astype(int)

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [7]:
movies_genres[['movieId', 'genre']].to_csv('items_genres.dat', index=False, sep='\t', header=False)
movies_tags[['movieId', 'tag']].to_csv('items_tags.dat', index=False, sep='\t', header=False)

### Divisão da base em treino e teste

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

### Exemplos de utilização do CaseRecommender

In [9]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10, as_similar_first=True).compute()
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_tags.dat', k_neighbors=10, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.340796 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.494514 sec
Eval:: MAE: 0.73347 RMSE: 0.964352 
[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.353146 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.631172 sec
Eval:: MAE: 0.765116 RMSE: 1.006911 


### ***Exercício 01:*** Verifique o efeito no RSME ao aumentar o número de vizinhos do algoritmo ItemAttributeKNN com usando gêneros. Explique.

In [10]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.441872 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.587597 sec
Eval:: MAE: 0.73347 RMSE: 0.964352 


In [11]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=100, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.456527 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 1.742719 sec
Eval:: MAE: 0.717937 RMSE: 0.945425 


In [12]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=1000, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.462729 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 3.796674 sec
Eval:: MAE: 0.695346 RMSE: 0.916212 


In [13]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10571 , as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.452284 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 3.474633 sec
Eval:: MAE: 0.695346 RMSE: 0.916212 


O RMSE reduz, o que faz sentido para uma recomendação item-based: ao avaliarmos a recomendação de um item i a um usuário u, conforme aumentamos o K, aumentamos os itens que podem ser considerados para fazer o cálculo. Porém, todos os itens considerados (vizinhos) já foram avaliados por u. Então supondo que o usuário é consistente em suas avaliações, considerar mais filmes, ou seja, aumentar o K, pode ser benéfico para reduzir o erro da predição de um usuário, ao custo de um aumento no tempo de predição.



### ***Exercício 02:*** Verifique o efeito na métrica MAP@1 ao aumentar o número de fatores do algoritmo BPRMF. Explique.

**Resposta**: Olhando os resultados, podemos observar uma queda no MAP@1 conforme aumentamos os fatores, mas essa queda se suaviza a partir de um certo ponto e volta a subir, até estagnar.

É dificil explicar a influência do número de fatores na qualidade da recomendação porque isso é dependente de quão variável são as notas no conjunto de treino/teste, já que um fator representa uma covariância de notas entre usuários. Aumentar o número de fatores pode ajudar a capturar padrões mais sútis nas notas, mas se as notas tiverem um comportamento muito simples de modelar, um número elevado de fatores pode trazer pouquíssimo retorno em qualidade. Ademais, um número muito elevado de fatores pode gerar matrizes de fatores altamente esparsas, o que contraria a principal vantagem de métodos de fatorização de matrizes, que é a redução de dimensionalidade. No exemplo observado, parece que os ganhos estagnaram a partir de quarenta fatores.

In [14]:
from caserec.recommenders.item_recommendation.bprmf import BprMF

In [15]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=1).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 129.622397 sec
prediction_time:: 2.357229 sec


Eval:: PREC@1: 0.378299 PREC@3: 0.262984 PREC@5: 0.224596 PREC@10: 0.173342 RECALL@1: 0.122623 RECALL@3: 0.241238 RECALL@5: 0.338066 RECALL@10: 0.509165 MAP@1: 0.378299 MAP@3: 0.460671 MAP@5: 0.469675 MAP@10: 0.444201 NDCG@1: 0.378299 NDCG@3: 0.539016 NDCG@5: 0.567817 NDCG@10: 0.56839 


In [16]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=3).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 111.169672 sec
prediction_time:: 2.411239 sec


Eval:: PREC@1: 0.3415 PREC@3: 0.243339 PREC@5: 0.206698 PREC@10: 0.162151 RECALL@1: 0.109704 RECALL@3: 0.223397 RECALL@5: 0.309068 RECALL@10: 0.475166 MAP@1: 0.3415 MAP@3: 0.426481 MAP@5: 0.436458 MAP@10: 0.417545 NDCG@1: 0.3415 NDCG@3: 0.508592 NDCG@5: 0.536576 NDCG@10: 0.544092 


MAP@1 com factors = 3: 0.38

In [17]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=6).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 124.409299 sec
prediction_time:: 2.472227 sec


Eval:: PREC@1: 0.352568 PREC@3: 0.268376 PREC@5: 0.229269 PREC@10: 0.174969 RECALL@1: 0.112611 RECALL@3: 0.247581 RECALL@5: 0.344466 RECALL@10: 0.515203 MAP@1: 0.352568 MAP@3: 0.449319 MAP@5: 0.459303 MAP@10: 0.438788 NDCG@1: 0.352568 NDCG@3: 0.540055 NDCG@5: 0.567672 NDCG@10: 0.569914 


MAP@1 com factors = 6: 0.36

In [18]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=9).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 119.841654 sec
prediction_time:: 2.501003 sec


Eval:: PREC@1: 0.36638 PREC@3: 0.277773 PREC@5: 0.235247 PREC@10: 0.178072 RECALL@1: 0.118773 RECALL@3: 0.257758 RECALL@5: 0.3584 RECALL@10: 0.526481 MAP@1: 0.36638 MAP@3: 0.465188 MAP@5: 0.473681 MAP@10: 0.451205 NDCG@1: 0.36638 NDCG@3: 0.559132 NDCG@5: 0.584128 NDCG@10: 0.582907 


MAP@1 com factors = 9: 0.36

In [19]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=12).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 131.616514 sec
prediction_time:: 2.803990 sec


Eval:: PREC@1: 0.359096 PREC@3: 0.274399 PREC@5: 0.232012 PREC@10: 0.176464 RECALL@1: 0.117339 RECALL@3: 0.254323 RECALL@5: 0.352463 RECALL@10: 0.521236 MAP@1: 0.359096 MAP@3: 0.459717 MAP@5: 0.469299 MAP@10: 0.447181 NDCG@1: 0.359096 NDCG@3: 0.5542 NDCG@5: 0.578914 NDCG@10: 0.579301 


MAP@1 com factors = 12; 0.37

In [20]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=40).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 127.352182 sec
prediction_time:: 2.457931 sec


Eval:: PREC@1: 0.404124 PREC@3: 0.299436 PREC@5: 0.248926 PREC@10: 0.184268 RECALL@1: 0.132319 RECALL@3: 0.278078 RECALL@5: 0.376083 RECALL@10: 0.543107 MAP@1: 0.404124 MAP@3: 0.499787 MAP@5: 0.503791 MAP@10: 0.477929 NDCG@1: 0.404124 NDCG@3: 0.589823 NDCG@5: 0.609593 NDCG@10: 0.606013 


In [21]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=100).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 124.837064 sec
prediction_time:: 2.480818 sec


Eval:: PREC@1: 0.426071 PREC@3: 0.305238 PREC@5: 0.253126 PREC@10: 0.18494 RECALL@1: 0.138031 RECALL@3: 0.281742 RECALL@5: 0.380121 RECALL@10: 0.541875 MAP@1: 0.426071 MAP@3: 0.515081 MAP@5: 0.517264 MAP@10: 0.490061 NDCG@1: 0.426071 NDCG@3: 0.601202 NDCG@5: 0.617874 NDCG@10: 0.613137 


### ***Exercício 03:*** Implemente a métrica Average Precision para um único usuário. Utilize as recomendações geradas pelo algoritmo BPRMF (consulte notebook Aula04_Exemplo como base).

In [22]:
recs = pd.read_csv('./ir_bprmf.dat', sep='\t', names=['userId', 'movieId', 'score'])
recs_user = recs.loc[(recs.userId==1), 'movieId'].tolist()
print(recs_user)

[19, 10, 14, 8, 4, 50, 5, 28, 56, 57]


In [23]:
label = test.loc[(test.userId==3), 'movieId'].tolist()

In [24]:
rec_list_test = recs.loc[(recs.userId==3), 'movieId'].tolist()

In [25]:
def precision_at_k(rec_list, ground_truth, k):
    focus_list = rec_list[:k]
    relevant_list = [1 if item in ground_truth else 0 for item in focus_list]
    n_relevants = sum(relevant_list)
    return n_relevants / k


In [26]:
def average_precision(user_id, recs_df, test_df, verbose=False):
    recs_user = recs_df.loc[(recs_df.userId==user_id), 'movieId'].tolist()
    ground_truth = test_df.loc[(test_df.userId==user_id), 'movieId'].tolist()

    precisions = 0
    relevant_list = [1 if item in ground_truth else 0 for item in recs_user]
    n_relevants = sum(relevant_list)
    
    if (n_relevants == 0):
        return 0

    num_recs = len(recs_user)
    for i in range(num_recs):
        if (relevant_list[i] == 1):
            # only consider relevant documents
            k=i+1
            precision = precision_at_k(recs_user, ground_truth,k=k)
            precisions += precision
            if (verbose):
                print(f"iter {i}; k={i+1}, list@k: {recs_user[:k]}, precision@k: {precision}; AP so far: {precisions/n_relevants}")
    return precisions/(n_relevants)


In [27]:
average_precision(3, recs, test, verbose=True)

iter 1; k=2, list@k: [12, 33], precision@k: 0.5; AP so far: 0.16666666666666666
iter 3; k=4, list@k: [12, 33, 18, 22], precision@k: 0.5; AP so far: 0.3333333333333333
iter 8; k=9, list@k: [12, 33, 18, 22, 13, 4, 28, 14, 23], precision@k: 0.3333333333333333; AP so far: 0.4444444444444444


0.4444444444444444

### ***Exercício 04:*** Um importante aspecto que pode ser avaliado em Sistemas de Recomendação é a diversidade da lista de recomendações. A métrica Intra-List Similarity (ILS) aplica uma função de similaridade (e.g. Cosseno, Jaccard, Pearson, etc.) entre todos os pares de itens da lista de recomendação, usando seus metadados como gêneros, tags, etc. Mais detalhes sobre essa métrica podem ser encontrados em: https://grouplens.org/site-content/uploads/Improving-WWW-20051.pdf

***a)*** Implemente uma função que calcula a ILS de uma lista de recomendação para um único usuário. Utilize os gêneros de filmes.

In [29]:
from itertools import combinations

In [30]:
def jaccard(list_a, list_b):
    set_a = set(list_a)
    set_b = set(list_b)
    inter_size = len(set_a.intersection(set_b))
    union_size = len(set_a.union(set_b))
    return inter_size / union_size


In [31]:
def get_genres_recommended_per_user(recs_df):
    genres_grouped = movies_genres.groupby(["movieId", "title"]).agg({"genre": list}).reset_index()
    recommendations_with_genres = recs_df.join(genres_grouped[["movieId", "genre"]], on="movieId", lsuffix='_left', rsuffix='_right').drop(columns=["movieId_right", "movieId_left"])
    return recommendations_with_genres.groupby("userId").agg({"genre": list}).reset_index()

In [32]:
genres_per_user = get_genres_recommended_per_user(recs)

In [33]:
def get_genres_recommended_for_user(userId, genres_per_user):
    return genres_per_user[genres_per_user["userId"] == userId].genre.item()

In [34]:
def ils(userId):
    genres_list = get_genres_recommended_for_user(userId, genres_per_user)
    sims = []
    for pair1, pair2 in combinations(genres_list, 2):
        sims.append(jaccard(pair1, pair2))
    return np.mean(sims)

In [35]:
ils(0)

0.1987301587301587

***b)*** Utilize a função que implementou no item (a) para calcular a ILS de todos os usuários da base de dados. Utilize as recomendações geradas pelo algoritmo BPR MF do CaseRecommender.

In [36]:
def get_ils_metric(target_df):
    df_copy = target_df.copy()
    df_copy["ILS"] = df_copy.apply(lambda r: ils(r.userId), axis=1)
    return df_copy

In [37]:
user_recs_with_ils = get_ils_metric(genres_per_user)

In [38]:
user_recs_with_ils

,userId,genre,ILS
0,0,"[[Crime, Mystery, Thriller], [Action, Crime, T...",0.198730
1,1,"[[Crime, Mystery, Thriller], [Adventure, Film-...",0.093704
2,2,"[[Action, Adventure, Mystery, Romance, Thrille...",0.085926
3,3,"[[Action, Adventure, Sci-Fi], [Action, Adventu...",0.125212
4,4,"[[Action, Adventure, Mystery, Romance, Thrille...",0.158466
...,...,...,...
11085,11085,"[[Action, Adventure, Sci-Fi], [Crime, Mystery,...",0.157249
11086,11086,"[[Action, Adventure, Sci-Fi], [Crime, Mystery,...",0.158889
11087,11087,"[[Documentary], [Action, Crime, Thriller], [Co...",0.148757
11088,11088,"[[Crime, Mystery, Thriller], [Comedy, Drama, R...",0.142884


***c)*** Aplique a função que implementou no item (b) para calcular a ILS das recomendações geradas pelos algoritmos ItemKNN e ItemAttributeKNN. Compare os resultados de todos os algoritmos, verificando inclusive o que acontece com a acurácia (precisão, MAP ou NDCG) quando a diversidade aumenta ou diminui.

**Resposta**: estranhamente, não observei diferença no ILS entre os métodos, mesmo o MAP sendo menor para o método BPR.

Em teoria, faz sentido existir um certo tradeoff entre ILS e metrica de acuracia, já que introduzir diversidade pode trazer a tona itens inesperados para o usuário, que não acha o item relevante. Calculando a correlação entre ILS e AP para BPR, observamos uma correlação ligeiramente negativa, o que faz sentido, mas é uma correlação basicamente espúria.

Como observado usuario a usuario, ou globalmente para cada metodo, não parece ter sido observado um comportamento claro de tradeoff entre ILS e AP ou ILS mediano e MAP.

Essa invariância de diversidade perante a métrica de precisão poderia ser explicada por não existir muita diversidade de gêneros na base de treino.



In [39]:
def create_AP_col(metrics_df, recs_df):
    df_copy = metrics_df.copy()
    df_copy['AP'] = metrics_df.apply(lambda r: average_precision(r.userId, recs_df=recs_df, test_df=test), axis=1)
    return df_copy

In [40]:
def MAP(metrics_df):
    return np.mean(metrics_df['AP'])

In [41]:
def median_ILS(metrics_df):
    return np.median(metrics_df['ILS'])

In [42]:
from caserec.recommenders.rating_prediction.itemknn import ItemKNN

Gerando dados

In [43]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=3).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 110.837018 sec
prediction_time:: 2.617985 sec


Eval:: PREC@1: 0.381421 PREC@3: 0.272349 PREC@5: 0.231123 PREC@10: 0.176133 RECALL@1: 0.124257 RECALL@3: 0.250707 RECALL@5: 0.348083 RECALL@10: 0.516215 MAP@1: 0.381421 MAP@3: 0.466441 MAP@5: 0.474827 MAP@10: 0.451298 NDCG@1: 0.381421 NDCG@3: 0.547925 NDCG@5: 0.57522 NDCG@10: 0.576019 


In [44]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', output_file='item_attribute_knn.dat', k_neighbors=3, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.685502 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.335413 sec
Eval:: MAE: 0.716087 RMSE: 0.94166 


In [45]:
ItemKNN('train.dat', 'test.dat', output_file='item_knn.dat', k_neighbors=3, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > ItemKNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.191296 sec
prediction_time:: 0.630935 sec
Eval:: MAE: 0.742235 RMSE: 0.982043 


Leitura

In [46]:
recs_item_attribute = pd.read_csv('./item_attribute_knn.dat', sep='\t', names=['userId', 'movieId', 'score'])

In [47]:
recs_item_attribute

,userId,movieId,score
0,0,0,3.863712
1,0,5,4.159026
2,1,13,3.631619
3,1,14,3.782424
4,1,16,3.188857
...,...,...,...
38120,11088,21,3.676758
38121,11089,12,4.391416
38122,11089,21,3.416535
38123,11089,57,2.899903


In [48]:
recs_item_knn = pd.read_csv('./item_knn.dat', sep='\t', names=['userId', 'movieId', 'score'])

In [49]:
recs_item_knn

,userId,movieId,score
0,0,0,4.499334
1,0,5,4.159026
2,1,13,4.891167
3,1,14,4.764759
4,1,16,3.885790
...,...,...,...
38120,11088,21,4.330219
38121,11089,12,5.000000
38122,11089,21,3.416535
38123,11089,57,2.899903


In [50]:
recs = pd.read_csv('./ir_bprmf.dat', sep='\t', names=['userId', 'movieId', 'score'])

Recupera o genero de cada filme

In [51]:
genres_per_user_item_attribute = get_genres_recommended_per_user(recs_item_attribute)

In [52]:
genres_per_user_item_knn = get_genres_recommended_per_user(recs_item_knn)

In [53]:
genres_per_user =  get_genres_recommended_per_user(recs)

Calcula o ILS e AP por usuário

In [54]:
item_attribute_metrics = create_AP_col(
                                metrics_df=get_ils_metric(genres_per_user_item_attribute), recs_df=recs_item_attribute
)

In [55]:
item_knn_metrics = create_AP_col(
                                metrics_df=get_ils_metric(genres_per_user_item_knn), recs_df=recs_item_knn
)

In [56]:
BPR_metrics = create_AP_col(
                                metrics_df=get_ils_metric(genres_per_user), recs_df=recs
)

In [57]:
BPR_metrics

,userId,genre,ILS,AP
0,0,"[[Crime, Mystery, Thriller], [Action, Crime, T...",0.233466,0.266667
1,1,"[[Crime, Mystery, Thriller], [Drama, Romance],...",0.222963,0.634921
2,2,"[[Adventure, Film-Noir, Sci-Fi, Thriller], [Ac...",0.085926,0.250000
3,3,"[[Action, Adventure, Sci-Fi], [Action, Crime, ...",0.151508,0.492857
4,4,"[[Action, Crime, Thriller], [Action, Adventure...",0.235185,0.111111
...,...,...,...,...
11085,11085,"[[Action, Adventure, Sci-Fi], [Crime, Mystery,...",0.157249,0.866667
11086,11086,"[[Action, Adventure, Sci-Fi], [Action, Adventu...",0.188148,0.111111
11087,11087,"[[Documentary], [Comedy, Drama, Romance, War],...",0.133915,0.777778
11088,11088,"[[Crime, Mystery, Thriller], [Comedy], [Comedy...",0.326296,0.277778


In [58]:
item_knn_metrics

,userId,genre,ILS,AP
0,0,"[[Adventure, Drama, Sci-Fi], [Action, Adventur...",0.233466,1.0
1,1,"[[Drama, Romance], [Comedy, Crime], [Action, C...",0.222963,1.0
2,2,"[[Drama, Mystery, Romance, Thriller]]",0.085926,1.0
3,3,"[[Action, Adventure, Sci-Fi, Thriller], [Actio...",0.151508,1.0
4,4,"[[Comedy, Drama]]",0.235185,1.0
...,...,...,...,...
10566,11085,"[[Adventure, Film-Noir, Sci-Fi, Thriller], [Ac...",0.157249,1.0
10567,11086,"[[Drama], [Comedy, Drama, Romance]]",0.188148,1.0
10568,11087,"[[Documentary], [Adventure, Film-Noir, Sci-Fi,...",0.133915,1.0
10569,11088,"[[Drama], [Comedy, Romance], [Drama, Mystery, ...",0.326296,1.0


In [59]:
item_attribute_metrics

,userId,genre,ILS,AP
0,0,"[[Adventure, Drama, Sci-Fi], [Action, Adventur...",0.233466,1.0
1,1,"[[Drama, Romance], [Comedy, Crime], [Action, C...",0.222963,1.0
2,2,"[[Drama, Mystery, Romance, Thriller]]",0.085926,1.0
3,3,"[[Action, Adventure, Sci-Fi, Thriller], [Actio...",0.151508,1.0
4,4,"[[Comedy, Drama]]",0.235185,1.0
...,...,...,...,...
10566,11085,"[[Adventure, Film-Noir, Sci-Fi, Thriller], [Ac...",0.157249,1.0
10567,11086,"[[Drama], [Comedy, Drama, Romance]]",0.188148,1.0
10568,11087,"[[Documentary], [Adventure, Film-Noir, Sci-Fi,...",0.133915,1.0
10569,11088,"[[Drama], [Comedy, Romance], [Drama, Mystery, ...",0.326296,1.0


In [60]:
MAP(item_attribute_metrics), median_ILS(item_attribute_metrics)

(1.0, 0.17415343915343917)

In [61]:
MAP(item_knn_metrics), median_ILS(item_knn_metrics)

(1.0, 0.17415343915343917)

In [62]:
MAP(BPR_metrics), median_ILS(BPR_metrics)

(0.43017751365065277, 0.17428571428571427)

In [63]:
BPR_metrics[["ILS", "AP"]].corr()

,ILS,AP
ILS,1.000000,-0.044418
AP,-0.044418,1.000000


In [64]:
item_attribute_metrics[["ILS", "AP"]].corr()

,ILS,AP
ILS,1.0,NaN
AP,NaN,NaN


In [65]:
item_knn_metrics[["ILS", "AP"]].corr()

,ILS,AP
ILS,1.0,NaN
AP,NaN,NaN
